In [10]:
from selenium import webdriver
from tempfile import mkdtemp
import sqlalchemy
import requests
import numpy as np
import pandas as pd
import datetime
import json
import urllib
from sqlalchemy import create_engine
from selenium_stealth import stealth
import pymysql
import undetected_chromedriver as uc
from sqlalchemy.engine.url import URL
from selenium.webdriver.common.by import By


URL_SEARCH = "https://www.tickpick.com/concerts/yeat-tickets/"

options = uc.ChromeOptions()
#options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument("--disable-gpu")
# options.add_argument("--window-size=1280x1696")
# options.add_argument("--single-process")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-dev-tools")
# options.add_argument("--no-zygote")
# options.add_argument(f"--user-data-dir={mkdtemp()}")
# options.add_argument(f"--data-path={mkdtemp()}")
# options.add_argument(f"--disk-cache-dir={mkdtemp()}")
# options.add_argument("--remote-debugging-port=9222")

#driver = webdriver.Chrome(r"C:\Users\Paul\Documents\chromedriver.exe", options=options)
driver = uc.Chrome(options=options)


driver.get(URL_SEARCH)

xpath_links = "//*[@id=\"events\"]/div[*]/a[1]"

links = []
while(not(links)):
    links = driver.find_elements_by_xpath(xpath_links)

venue_names = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[2]/span/span")
dates = driver.find_elements_by_xpath("//*[@id=\"events\"]/div[*]/div[1]/span[1]")

links = list(map(lambda elem: elem.get_attribute("href"), links))
venue_names = list(map(lambda elem: elem.text, venue_names))
dates = list(map(lambda elem: elem.text, dates))
print(links)

def add_page_to_row_list(link, row_list, driver, venue_name, date, time):
    driver.get(link)

    xpath = "//*[@id=\"listingContainer\"]/div[*]"

    elements = []

    while len(elements) == 0:
        elements = driver.find_elements_by_xpath(xpath)
        

    for elem in elements:
        dic = {}
        dic["Date"] = time
        dic["VenueID"] = venue_name
        dic["DateEvent"] = date
        dic["Quantity"] = elem.find_element_by_tag_name("select").text.split()[0]
        
        print(venue_name)
        
        price = ""
        while len(price) == 0:
            price = elem.find_element(by=By.TAG_NAME, value="b").text
        dic["Price"] = price[1:]
        row_list.append(dic)

row_list = []

date = datetime.datetime.now()
time = date.strftime("%b %d %I:%M%p")

for i in range(len(links)):
    print(links[i])
    add_page_to_row_list(links[i], row_list, driver, venue_names[i], dates[i], time)

df = pd.DataFrame(row_list)
df.head()

df["Price"].head(100)

server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick_database'
username = 'pauldoucet'
driver = '{ODBC Driver 17 for SQL Server}'
ca_path = "DigiCertGlobalRootCA.crt.pem"

f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)

df.to_sql("test_table", con=engine, if_exists="append", index=False)

<ipython-input-10-22a85d4a21bb>:51: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  price = driver.find_element_by_xpath("//*[@id=\"listingContainer\"]/div[1]/label/b").text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="listingContainer"]/div[1]/label/b"}
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B97413+2389011]
	Ordinal0 [0x00B29F61+1941345]
	Ordinal0 [0x00A1C658+837208]
	Ordinal0 [0x00A491DD+1020381]
	Ordinal0 [0x00A4949B+1021083]
	Ordinal0 [0x00A76032+1204274]
	Ordinal0 [0x00A64194+1130900]
	Ordinal0 [0x00A74302+1196802]
	Ordinal0 [0x00A63F66+1130342]
	Ordinal0 [0x00A3E546+976198]
	Ordinal0 [0x00A3F456+980054]
	GetHandleVerifier [0x00D49632+1727522]
	GetHandleVerifier [0x00DFBA4D+2457661]
	GetHandleVerifier [0x00C2EB81+569713]
	GetHandleVerifier [0x00C2DD76+566118]
	Ordinal0 [0x00B30B2B+1968939]
	Ordinal0 [0x00B35988+1989000]
	Ordinal0 [0x00B35A75+1989237]
	Ordinal0 [0x00B3ECB1+2026673]
	BaseThreadInitThunk [0x7647FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76EC7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x76EC7A4E+238]


In [11]:
from sqlalchemy.engine.url import URL

f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)

df.to_sql("test_table", con=engine, if_exists="append", index=False)

In [13]:
df.head(100)

,Date,VenueID,DateEvent,Quantity,Price
0,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,2,
1,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,2,
2,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,4,76
3,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,2,79
4,04.06.22_19.09,"Concord Music Hall - Chicago, IL",Apr 07,4,88
...,...,...,...,...,...
95,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,8,
96,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,8,
97,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,8,
98,04.06.22_19.09,"The Summit Music Hall - Denver, CO",Apr 15,2,


In [8]:
from sqlalchemy import delete
server = 'tickpick-scraper-database.mysql.database.azure.com'
database = 'tickpick_database'
username = 'pauldoucet'
driver = '{ODBC Driver 17 for SQL Server}'
ca_path = "DigiCertGlobalRootCA.crt.pem"
f = open("password.json")
data = json.load(f)
password = data["password"]

sqlUrl = URL(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host=server,
    port=3306,
    database=database,
    query={"ssl_ca": ca_path},
)
engine = create_engine(sqlUrl)
sql = "DELETE FROM test_table WHERE date='04.06.22_19.09' OR date='04.06.22_20.29'"
 
result = engine.execute(sql)